In [3]:
import pandas as pd
import os
import re

# ============================================================
# 🧭 Y14_경찰청 치안센터 주소 전처리 파이프라인 (서울)
# - 파일 로드
# - 서울 데이터만 필터링
# - 주소에서 '지역구' 추출
# - 지역구별 치안센터 수 수치화
# - 오름차순 정렬 및 저장
# ============================================================

# ===== 0) 경로 설정 =====
relative_data_path = '../../data/raw/'
file_name = 'Y14_경찰청_전국 치안센터 주소 현황_20250630_TM.csv'
file_path = os.path.join(relative_data_path, file_name)

output_dir = '../../data/preprocessing'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(
    output_dir,
    'Y14_전처리_치안센터_설치수_20250630기준.csv'
)

    # ===== 1) 파일 로드 =====
df = pd.read_csv(file_path, sep='\t', encoding='utf-8-sig')
print(f"\n✅ [1단계] 파일 로드 완료: {df.shape[0]:,}행 × {df.shape[1]}열")
print("📌 컬럼명:", df.columns.tolist())
df.info()


✅ [1단계] 파일 로드 완료: 741행 × 18열
📌 컬럼명: ['연번', '시도청', '경찰서', '치안센터명', '주소', '입력주소', 'X', 'Y', 'CLSS', 'PNU', '주소구분', '표준신주소', '표준구주소', '우편번호', '행정동코드', '행정동명', '법정동코드', '법정동명']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 741 entries, 0 to 740
Data columns (total 18 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      741 non-null    int64  
 1   시도청     741 non-null    object 
 2   경찰서     741 non-null    object 
 3   치안센터명   741 non-null    object 
 4   주소      741 non-null    object 
 5   입력주소    741 non-null    object 
 6   X       741 non-null    int64  
 7   Y       741 non-null    int64  
 8   CLSS    741 non-null    object 
 9   PNU     711 non-null    float64
 10  주소구분    741 non-null    object 
 11  표준신주소   680 non-null    object 
 12  표준구주소   715 non-null    object 
 13  우편번호    666 non-null    float64
 14  행정동코드   688 non-null    float64
 15  행정동명    688 non-null    object 
 16  법정동코드   687 non-null    float64
 17  법정동명    687 n

In [4]:
# ===== 2) 서울 데이터만 필터링 =====
df_seoul = df[df['주소'].str.contains('서울', na=False)].copy()
print(f"✅ [2단계] 서울 데이터 필터링 완료: {df_seoul.shape[0]:,}행")

# ===== 3) 주소에서 '지역구' 추출 =====
def extract_district(address):
    if pd.isna(address):
        return None
    match = re.search(r'([가-힣]+구)', str(address))
    return match.group(1) if match else None

df_seoul['지역구'] = df_seoul['주소'].apply(extract_district)

print("\n✅ [3단계] 주소에서 지역구 추출 완료")
print(df_seoul[['주소', '지역구']].head(10))
print(f"📊 서울 지역구 결측치 수: {df_seoul['지역구'].isnull().sum()}")

# ===== 4) 지역구별 치안센터 수 수치화 =====
district_counts = df_seoul['지역구'].value_counts().sort_index()
df_grouped = district_counts.reset_index()
df_grouped.columns = ['지역구', '지역구별 치안센터 수']

print("\n✅ [4단계] 지역구별 치안센터 수 계산 완료")
print(df_grouped.head(10))

# ===== 5) 결과 저장 =====
df_grouped.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n💾 [5단계] '{os.path.basename(output_file)}' 저장 완료")
print(f"📊 최종 행 수: {len(df_grouped):,} 행")
print("\n📌 저장 컬럼:", df_grouped.columns.tolist())
print(df_grouped.head(10))


✅ [2단계] 서울 데이터 필터링 완료: 53행

✅ [3단계] 주소에서 지역구 추출 완료
                    주소   지역구
0       서울 중구 충무로2길 39    중구
1        서울 종로구 비봉길 13   종로구
2  서울 서대문구 북아현로 59-1번지  서대문구
3       서울 종로구 돈화문로 28   종로구
4     서울 용산구 청파로 274-2   용산구
5        서울 용산구 신흥로 90   용산구
6      서울 용산구 이태원로 281   용산구
7     서울 용산구 이촌로71길 24   용산구
8     서울 성북구 보문로29길 92   성북구
9   서울 성북구 정릉로 31길 30    성북구
📊 서울 지역구 결측치 수: 2

✅ [4단계] 지역구별 치안센터 수 계산 완료
    지역구  지역구별 치안센터 수
0   강남구            2
1   강북구            3
2   강서구            2
3   광진구            2
4   구로구            5
5   금천구            2
6   노원구            1
7   도봉구            1
8  동대문구            1
9   동작구            2

💾 [5단계] 'Y14_전처리_치안센터_설치수_20250630기준.csv' 저장 완료
📊 최종 행 수: 23 행

📌 저장 컬럼: ['지역구', '지역구별 치안센터 수']
    지역구  지역구별 치안센터 수
0   강남구            2
1   강북구            3
2   강서구            2
3   광진구            2
4   구로구            5
5   금천구            2
6   노원구            1
7   도봉구            1
8  동대문구            1
9   동작구            2
